<a href="https://colab.research.google.com/github/diptivs/machine_learning/blob/master/Market_Basket_Item.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Load test and train data

In [0]:
import pandas as pd
# Setup google auth
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# load the file
#https://drive.google.com/file/d/15unl1fPYvwkeafjL_th3EN75NtE4zNLW/view?usp=sharing --> train data
#https://drive.google.com/open?id=1Mqgd6bLTW-NBMY-hB6xwJvDd40HoWq3v --> test data

#load train data
file_id = '15unl1fPYvwkeafjL_th3EN75NtE4zNLW'
downloaded = drive.CreateFile({'id':file_id}) 
downloaded.GetContentFile('TRAIN-ARULES.csv')  
data_in = pd.read_csv('TRAIN-ARULES.csv')
data_in.head()

#load test data
file_id = '1Mqgd6bLTW-NBMY-hB6xwJvDd40HoWq3v'
downloaded = drive.CreateFile({'id':file_id}) 
downloaded.GetContentFile('testarules.csv')  
test_in = pd.read_csv('testarules.csv')
test_in.head()


,Item1,Item2,Item3,Item4,Item5
0,Dark Chocolate Minis,Organic Pink Lemonade Bunny Fruit Snacks,Peach-Pear Sparkling Water,NaN,NaN
1,Banana,Dry Roasted Almonds,NaN,NaN,NaN


## 2. After loading data, we need to consolidate the items into 1 transaction per row with each order_id+user_id

In [0]:
gb = data_in.groupby(('order_id','user_id'))
result = gb['product_name'].unique()
#print(result)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


## 3. Create List of records

In [0]:
cnt = result.shape[0]
records = []

for i in range(cnt):
  records.append(result[i])

#print(records)

##4. Create List of Test Data

In [0]:
print(test_in.shape)
checklist = [[]]
checklist.clear()
rowlist = []
for i in range(test_in.shape[0]):
  for j in range(test_in.shape[1]):
    if(pd.isnull(test_in.iloc[i,j]) != True):
      rowlist.append(test_in.iloc[i,j])
  if(rowlist != []):
    if(checklist.count != 0):
      checklist.append(rowlist)
    else:
      checklist=rowlist
    rowlist = []

print(checklist)

(2, 5)
[['Dark Chocolate Minis', 'Organic Pink Lemonade Bunny Fruit Snacks', 'Peach-Pear Sparkling Water'], ['Banana', 'Dry Roasted Almonds']]


#Using apyori Library

In [0]:
!pip install apyori
import pandas as pd
from apyori import apriori

## 5. Apply apriori to generate association rules

In [0]:
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2)
association_results = list(association_rules)

print(len(association_results))

print(association_results[0])

1015
RelationRecord(items=frozenset({'Apples', '0% Greek Strained Yogurt'}), support=0.006346967559943582, ordered_statistics=[OrderedStatistic(items_base=frozenset({'0% Greek Strained Yogurt'}), items_add=frozenset({'Apples'}), confidence=0.6428571428571428, lift=41.43506493506493), OrderedStatistic(items_base=frozenset({'Apples'}), items_add=frozenset({'0% Greek Strained Yogurt'}), confidence=0.40909090909090906, lift=41.43506493506493)])


## 6. Predict item3 and item4 for test data

In [0]:
anslist = []
finallist = [[]]
finallist.clear()

for l in range(len(checklist)):
  print('\x1b[1;33m'+"Test Data[{}]={}".format(l,checklist[l])+'\x1b[0m')
  for i in range(len(association_results)):
    list_items = list(association_results[i][0])
    if(len(list_items) == 5):
      if(checklist[l][0] in list_items and (checklist[l][1] in list_items) and (checklist[l][2] in list_items)):
        for j in range(5):
          if(list_items[j] not in checklist[l]):
            if(list_items[j] not in anslist):
              anslist.append(list_items[j])
    if(anslist != []):
      if(finallist.count == 0):
        finallist = anslist
      else:
        finallist.append(anslist)
      anslist=[]

  for k in range(len(finallist)):
    if(len(finallist[k])>1):
      print('\x1b[1;31m'+"item3={} and item4={}".format(finallist[k][0], finallist[k][1])+'\x1b[0m')


Test Data[0]=['Dark Chocolate Minis', 'Organic Pink Lemonade Bunny Fruit Snacks', 'Peach-Pear Sparkling Water']
item3=Organic Heritage Flakes Cereal and item4=Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars
item3=Organic Heritage Flakes Cereal and item4=Organic Graham Crunch Cereal
item3=Organic Graham Crunch Cereal and item4=Sparkling Water, Natural Mango Essenced
item3=Organic Heritage Flakes Cereal and item4=Sparkling Water, Natural Mango Essenced
Test Data[1]=['Banana', 'Dry Roasted Almonds']
item3=Organic Heritage Flakes Cereal and item4=Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars
item3=Organic Heritage Flakes Cereal and item4=Organic Graham Crunch Cereal
item3=Organic Graham Crunch Cereal and item4=Sparkling Water, Natural Mango Essenced
item3=Organic Heritage Flakes Cereal and item4=Sparkling Water, Natural Mango Essenced


#Using mlxtend.frequent_patterns Library

In [0]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

## 5. Apply apriori to generate association rules

In [0]:
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.0045, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.009873,(0% Greek Strained Yogurt)
1,0.004937,"(100% Juice, Variety Pack)"
2,0.009168,(100% Premium Select Not From Concentrate Pure...
3,0.005642,(100% Recycled Paper Towels)
4,0.014810,(1500 Pale Ale)
5,0.007757,(2% Reduced Fat Milk)
6,0.009168,(49 Flavors Jelly Belly Jelly Beans)
7,0.009168,(80 Vodka Holiday Edition)
8,0.008463,(85% Lean Ground Beef)
9,0.004937,(Acai Berry Chia Bar)


In [0]:
from mlxtend.frequent_patterns import association_rules

rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

## 6. Predict item3 and item4 for test data

In [0]:
pd.set_option('max_colwidth', 200)
for k in range(len(checklist)):
  print('\x1b[1;33m'+"Test Data[{}]={}".format(k,checklist[k])+'\x1b[0m')
  lst=checklist[k]
  res_dct = {lst[i]for i in range(0, len(lst))} 
  ans = rules[rules['antecedents'] == res_dct]
  print("Next item predictions:\n{}".format(ans['consequents']))

Test Data[0]=['Dark Chocolate Minis', 'Organic Pink Lemonade Bunny Fruit Snacks', 'Peach-Pear Sparkling Water']
Next item predictions:
3699                                       (Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars)
3752                                                                             (Organic Graham Crunch Cereal)
3796                                                                           (Organic Heritage Flakes Cereal)
3838                                                                  (Sparkling Water, Natural Mango Essenced)
4908       (Organic Heritage Flakes Cereal, Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars)
4938                                             (Organic Heritage Flakes Cereal, Organic Graham Crunch Cereal)
5023                                    (Sparkling Water, Natural Mango Essenced, Organic Graham Crunch Cereal)
5055                                  (Sparkling Water, Natural Mango Essenced, O